# Ансамбль моделей. Классификация меланоцитных заболеваний и классификация из меланоцитных меланом и невусов 

## Предобработка данных

Импорт библиотек

In [7]:
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
from dvclive import Live
import csv
import glob
from tqdm import tqdm
from datasets import DatasetDict, Dataset

Функция изменения размера изображений в соответствии со входным размером модели

In [2]:
def resize_and_normalize(image_path, output_path, size=(224, 224), format='JPEG'):
    image = Image.open(image_path)
    image = image.resize(size)
    if image.mode in ['RGBA', 'P']:
        image = image.convert('RGB')
    image.save(output_path, format=format)

In [3]:
def preprocess_dataset(input_dir, output_dir, csv_path, test_size=0.2):
    os.makedirs(output_dir, exist_ok=True)
    data = pd.read_csv(csv_path)

    # Create directories for each label
    for label in data['diagnosis'].unique():
        if label == 'MEL':
            os.makedirs(os.path.join(output_dir, 'train', label,"MEL"), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'train', label,"NV"), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'test', label,"NV"), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'test', label,"MEL"), exist_ok=True)
        else:
            os.makedirs(os.path.join(output_dir, 'train', label), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'test', label), exist_ok=True)


    # Split data into training and testing
    train_df, test_df = train_test_split(data, test_size=test_size,random_state=22)


    # Process train images with a progress bar
    for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc='Processing train images'):
        image_path = os.path.join(input_dir, row['image_name'] + '.jpg')
        if str(row['melanocit']) != 'nan':
            if os.path.exists(image_path):
                output_path = os.path.join(output_dir, 'train', row['diagnosis'], str(row['melanocit']), row['image_name'] + '.jpg')
                resize_and_normalize(image_path, output_path)
        else:
            if os.path.exists(image_path):
                output_path = os.path.join(output_dir, 'train', row['diagnosis'], row['image_name'] + '.jpg')
                resize_and_normalize(image_path, output_path)

    # Process test images with a progress bar
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc='Processing test images'):
        image_path = os.path.join(input_dir, row['image_name'] + '.jpg')
        if str(row['melanocit']) != 'nan':
            if os.path.exists(image_path):
                output_path = os.path.join(output_dir, 'test', row['diagnosis'], str(row['melanocit']), row['image_name'] + '.jpg')
                resize_and_normalize(image_path, output_path)
        else:
            if os.path.exists(image_path):
                output_path = os.path.join(output_dir, 'test', row['diagnosis'], row['image_name'] + '.jpg')
                resize_and_normalize(image_path, output_path)

In [5]:
input_dir = "/home/jovyan/ensemble-of-models/data/raw/classification/isic2019_train"
output_dir = "/home/jovyan/ensemble-of-models/data/preprocessed"
csv_path = "/home/jovyan/ensemble-of-models/data/raw/classification/ans_isic.csv"
preprocess_dataset(input_dir, output_dir, csv_path)

Processing test images: 100%|██████████| 5067/5067 [00:43<00:00, 115.55it/s]
	src/pipeline.ipynb


## Обучение моделей

Импорт необходимых библиотек

In [6]:
import os
import argparse
import numpy as np
import yaml
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer, DefaultDataCollator
from transformers import ViTImageProcessor, ViTConfig
from datasets import Dataset, load_metric
from dvclive import Live
from transformers.integrations import DVCLiveCallback
import tqdm
import torch
from datasets import DatasetDict
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    GaussianBlur,
    ToTensor,
    RandomRotation,
)
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
import evaluate
from PIL import Image
import glob
import yaml

2024-11-04 11:22:51.363700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730719371.376981 3608098 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730719371.380986 3608098 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-04 11:22:51.397210: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Класс тренировки моделей.

In [9]:
class TrainModel:
    def __init__(self, label_map, model, output_path, epoch, batch_size, data_path):
        self.LABEL_MAP = label_map
        self.model = model
        self.output_path = output_path
        self.epoch = epoch
        self.batch_size = batch_size
        self.data_path = data_path

    def train_model(self):
        def cust_load_model(data_dir): #кастомный способ загрузки датасета 
            dataset = DatasetDict()
            
            for train_test in os.listdir(data_dir):
                train_test_path = os.path.join(data_dir, train_test)
                
                if not os.path.isdir(train_test_path):
                    continue 
                
                image_paths = [] 
                class_labels = [] 
                subclass_labels = [] 
                
                for class_dir in os.listdir(train_test_path):
                    class_path = os.path.join(train_test_path, class_dir) 
                    
                    if not os.path.isdir(class_path):
                        continue 
                    
                    if class_dir == 'MEL':
                        for subclass_dir in os.listdir(class_path): 
                            subclass_path = os.path.join(class_path, subclass_dir) 
                            if os.path.isdir(subclass_path): 
                                for image_file in glob.glob(os.path.join(subclass_path, '*.jpg')):
                                    image_paths.append(Image.open(image_file))  
                                    class_labels.append(class_dir)
                                    subclass_labels.append(subclass_dir) 
                    else:
                        for image_file in glob.glob(os.path.join(class_path, '*.jpg')):
                            image_paths.append(Image.open(image_file))  
                            class_labels.append(class_dir)
                            subclass_labels.append(class_dir) 

                dat = Dataset.from_dict({
                    'image': image_paths,
                    'class_label': class_labels,
                    'subclass_label': subclass_labels
                })
                dataset[train_test] = dat 

            return dataset

        def melanocit_preprocess(example): #обработка для классификатора меланоцитов
            return{
                'image' : example['image'],
                'label' : example['class_label']
            }
        def melanoma_preprocess(example): #обработка для классификатора меланом и невусов
            return{
                'image' : example['image'],
                'label' : example['subclass_label']
            }
        train_dataset = cust_load_model(self.data_path)
        if list(self.LABEL_MAP.keys()) == ['MEL','NOTMEL']:
            train_dataset['train'] = train_dataset['train'].map(melanocit_preprocess)
            train_dataset['test'] = train_dataset['test'].map(melanocit_preprocess)
            train_dataset = train_dataset.class_encode_column("label")
        else:
            train_dataset['train'] = train_dataset['train'].map(melanoma_preprocess)
            train_dataset['test'] = train_dataset['test'].map(melanoma_preprocess)
            train_dataset = train_dataset.class_encode_column("label")

        image_processor  = AutoImageProcessor.from_pretrained(self.model)

        normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
        if "height" in image_processor.size:
            size = (image_processor.size["height"], image_processor.size["width"])
            crop_size = size
            max_size = None
        elif "shortest_edge" in image_processor.size:
            size = image_processor.size["shortest_edge"]
            crop_size = (size, size)
            max_size = image_processor.size.get("longest_edge")
        train_transforms = Compose(
                [
                    Resize(size),
                    ToTensor(),
                    normalize,
                ]
            )

        val_transforms = Compose(
                [
                    Resize(size),
                    ToTensor(),
                    normalize,
                ]
            )
        def preprocess_train(example_batch):
            """Apply train_transforms across a batch."""
            example_batch["pixel_values"] = [
                train_transforms(image.convert("RGB")) for image in example_batch["image"]
            ]
            return example_batch

        def preprocess_val(example_batch):
            """Apply val_transforms across a batch."""
            example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
            return example_batch

        train_dataset['train'].set_transform(preprocess_val)
        train_dataset['test'].set_transform(preprocess_val)

        acc_metric = evaluate.load("accuracy", trust_remote_code=True)
        f1_metric = evaluate.load("f1", trust_remote_code=True)

        def collate_fn(examples):
            pixel_values = torch.stack([example["pixel_values"] for example in examples])
            labels = torch.tensor([example["label"] for example in examples])
            return {"pixel_values": pixel_values, "labels": labels}

        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)
            accuracy = acc_metric.compute(predictions=predictions, references=labels)
            f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')
            return {"accuracy": accuracy['accuracy'], "f1": f1['f1']}

        labels = train_dataset['train'].features["label"].names
        label2id, id2label = dict(), dict()
        for i, label in enumerate(labels):
            label2id[label] = i
            id2label[i] = label


        model = AutoModelForImageClassification.from_pretrained(
        self.model, 
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
        )

        args = TrainingArguments(
        self.output_path,
        remove_unused_columns=False,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=5e-5,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        warmup_ratio=0.1,
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        push_to_hub=False,
        per_device_train_batch_size=self.batch_size,
        num_train_epochs=self.epoch,
        )   
        trainer = Trainer(
        model,
        args,
        train_dataset=train_dataset['train'],
        eval_dataset=train_dataset['test'],
        tokenizer=image_processor,
        compute_metrics=compute_metrics,
        data_collator=collate_fn,
        )
        trainer.train()
        trainer.save_model(self.output_path)

Обучение модели

In [12]:
def start_train(output_path,data_path,params_file):
    path_model_1 = output_path + '/model_mel_notmel.pkl'
    path_model_2 = output_path + '/model_mel_nv.pkl'
    with open(params_file, 'r') as f:
        params = yaml.safe_load(f)

    TrainModel(params['model_mel_notmel']["label_map"],params['model_mel_notmel']["arch"], path_model_1,params['model_mel_notmel']['train']["epochs"], params['model_mel_notmel']['train']["batch_size"],data_path).train_model()
    TrainModel(params['model_mel_nv']["label_map"],params['model_mel_nv']["arch"], path_model_2, params['model_mel_nv']['train']["epochs"], params['model_mel_nv']['train']["batch_size"],data_path).train_model()

In [13]:
output_path = "/home/jovyan/ensemble-of-models/models"
data_path = "/home/jovyan/ensemble-of-models/data/preprocessed"
params_file = "/home/jovyan/ensemble-of-models/params.yaml"
start_train(output_path, data_path, params_file)

Casting to class labels: 100%|██████████| 19547/19547 [00:00<00:00, 720629.87 examples/s]
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-tiny-patch4-window16-256 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.291600,0.270321,0.885376,0.857804


	src/pipeline.ipynb
Casting to class labels: 100%|██████████| 19547/19547 [00:00<00:00, 719788.42 examples/s]
/home/jovyan/.local/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.418700,0.452547,0.824801,0.781603


	src/pipeline.ipynb


## Оценка моделей

Импорт библиотек

In [1]:
import os
import argparse
import yaml, json
import numpy as np
from transformers import AutoModelForImageClassification, ViTFeatureExtractor, pipeline, ViTImageProcessor
import datasets
from datasets import load_metric, load_dataset
import dvclive
import evaluate
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from markdownwriter import MarkdownWriter
import pandas as pd
from markdownwriter import *
import seaborn as sns

/home/jovyan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-07 13:27:31.265715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730986051.278360 3818452 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730986051.282009 3818452 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 13:27:31.296958: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operatio

In [2]:
def load_model(model_path):
    model = AutoModelForImageClassification.from_pretrained(model_path)
    return model

def save_classification_report(live, references, predictions, labels):
    # Generate the classification report as a dictionary
    report = classification_report(references, predictions, target_names=labels, output_dict=True)
    
    # Specify the path for the JSON file
    report_path = "classification_report.json"
    
    # Serialize the report dictionary to a JSON formatted string and save it to a file
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=4)  # `indent=4` for pretty-printing
    
    # Log the JSON file as an artifact with DVCLive
    live.log_artifact(report_path)

In [3]:
def save_confusion_matrix(live, references, predictions, labels):
    cm = confusion_matrix(references, predictions, labels=list(LABEL_MAP.values()))
    plt.figure(figsize=(10, 7))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig("confusion_matrix.png")
    plt.close()
    live.log_artifact("confusion_matrix.png")

Кастомный способ загрузки датасета для предобработки

In [4]:
def cust_load_model(data_dir): #кастомный способ загрузки датасета 
    dataset = DatasetDict()

    for train_test in os.listdir(data_dir):
        train_test_path = os.path.join(data_dir, train_test)
        
        if not os.path.isdir(train_test_path):
            continue
        
        image_paths = [] 
        class_labels = [] 
        subclass_labels = [] 
        
        for class_dir in os.listdir(train_test_path):
            class_path = os.path.join(train_test_path, class_dir) 
            
            if not os.path.isdir(class_path):
                continue 
            
            if class_dir == 'MEL':
                for subclass_dir in os.listdir(class_path): 
                    subclass_path = os.path.join(class_path, subclass_dir) 
                    
                    if os.path.isdir(subclass_path): 
                        for image_file in glob.glob(os.path.join(subclass_path, '*.jpg')):
                            image_paths.append(Image.open(image_file))
                            class_labels.append(class_dir)
                            subclass_labels.append(subclass_dir) 
            else:
                for image_file in glob.glob(os.path.join(class_path, '*.jpg')):
                    image_paths.append(Image.open(image_file))
                    class_labels.append(class_dir)
                    subclass_labels.append(class_dir) 

        dat = Dataset.from_dict({
            'image': image_paths,
            'class_label': class_labels,
            'subclass_label': subclass_labels
        })
        dataset[train_test] = dat
    return dataset

In [5]:
def evaluate_model(data_path, model_path, params_file):
    # Load parameters and model
    with open(params_file, 'r') as f:
        params = yaml.safe_load(f)
        
    
    model1 = load_model(model_path+'model_mel_notmel.pkl')
    model2 = load_model(model_path+'model_mel_nv.pkl')
    feature_extractor1 = ViTImageProcessor.from_pretrained(model_path+'model_mel_notmel.pkl')
    feature_extractor2 = ViTImageProcessor.from_pretrained(model_path+'model_mel_nv.pkl')

    # Prepare data and evaluation pipeline
    full_dataset = cust_load_model(data_path)
    test_dataset = full_dataset["test"]
    
    eval_pipeline1 = pipeline("image-classification", model=model1, feature_extractor=feature_extractor1)
    eval_pipeline2 = pipeline("image-classification", model=model2, feature_extractor=feature_extractor2)

    # Collect predictions and references with a progress bar
    predictions1 = []
    references1 = []
    predictions2 = []
    references2 = []
    label_map_1 = params['model_mel_notmel']["label_map"]
    label_map_2 = params['model_mel_nv']["label_map"]

    for example in tqdm(test_dataset, desc="Processing Images", leave=True):
        result1 = eval_pipeline1(example["image"])
        prediction_label1 = result1[0]['label'].split('_')[-1]
        prediction1 = label_map_1.get(prediction_label1,prediction_label1)
        reference1 = label_map_1.get(example['class_label'],example['class_label'])
        predictions1.append(prediction1)
        references1.append(reference1)
        if prediction_label1=='MEL':
            result2 = eval_pipeline2(example['image'])
            prediction_label2 = result2[0]['label'].split('_')[-1]
            prediction2 = label_map_2.get(prediction_label2,prediction_label2)
            reference2 = label_map_2.get(example['subclass_label'],example['subclass_label'])
            predictions2.append(prediction2)
            references2.append(reference2)


    f1_m1 = f1_score(references1, predictions1, average='macro')  # You can change average to 'micro', 'weighted', or None
    precision_m1 = precision_score(references1, predictions1, average='macro')
    accuracy_m1 = accuracy_score(references1, predictions1)

    f1_m2 = f1_score(references2, predictions2, average='macro')  # You can change average to 'micro', 'weighted', or None
    precision_m2 = precision_score(references2, predictions2, average='macro')
    accuracy_m2 = accuracy_score(references2, predictions2)

    print(f"F1_model_1 Score (Macro): {f1_m1}")
    print(f"F1_model_2 Score (Macro): {f1_m2}")
    print(f"accuracy_model_1 (Macro): {accuracy_m1}")
    print(f"accuracy_model_2 (Macro): {accuracy_m2}")

    # Classification report and confusion matrix model1
    cr1 = classification_report(references1, predictions1, target_names=list(label_map_1.keys()), zero_division=0, output_dict=True)
    cm1 = confusion_matrix(references1, predictions1, labels=list(label_map_1.values()))
    
    # Classification report and confusion matrix model2
    cr2 = classification_report(references2, predictions2, target_names=list(label_map_2.keys()), zero_division=0, output_dict=True)
    cm2 = confusion_matrix(references2, predictions2, labels=list(label_map_2.values()))
    
    # Convert classification report and confusion matrix to DataFrame for better formatting
    report_df1 = pd.DataFrame(cr1).transpose()
    matrix_df1 = pd.DataFrame(cm1)

    report_df2 = pd.DataFrame(cr2).transpose()
    matrix_df2 = pd.DataFrame(cm2)

    print("Report DataFrame 1:")
    print(report_df1)
    print("\nMatrix DataFrame 1:")
    print(matrix_df1)
    print("\nReport DataFrame 2:")
    print(report_df2)
    print("\nMatrix DataFrame 2:")
    print(matrix_df2)
    cm1 = confusion_matrix(references1, predictions1)
    cm2 = confusion_matrix(references2, predictions2)

    plt.figure(figsize=(12, 10))

    plt.subplot(1, 2, 1)
    sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues',
                xticklabels=list(label_map_1.values()),
                yticklabels=list(label_map_1.values()))
    plt.title('Confusion Matrix Model 1')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

    plt.subplot(1, 2, 2)
    sns.heatmap(cm2, annot=True, fmt='d', cmap='Blues',
                xticklabels=list(label_map_2.values()),
                yticklabels=list(label_map_2.values()))
    plt.title('Confusion Matrix Model 2')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

    plt.tight_layout()
    plt.show()

In [8]:
data_path = "/home/jovyan/ensemble-of-models/data/preprocessed"
model_path = "/home/jovyan/ensemble-of-models/models/"
params_file = "/home/jovyan/ensemble-of-models/params.yaml"
evaluate_model(data_path, model_path, params_file)

Processing Images:   0%|          | 5/4903 [00:15<4:00:30,  2.95s/it]

Processing Images:   0%|          | 6/4903 [00:19<4:21:52,  3.21s/it]

In [ ]:
md_writer = MarkdownWriter(f"report/report_ans.md")
#md_writer.header1('Report')

md_writer.header2('Params')

md_writer.print_config(params)

train_description = describe_dataset('data/preprocessed/train/')
test_description = describe_dataset('data/preprocessed/test/')
md_writer.describe_dataset_markdown(train_description, "Training Dataset Description")
md_writer.describe_dataset_markdown(test_description, "Testing Dataset Description")

md_writer.header2('Metrics')
md_writer.print_data(f"F1 Score (Macro) Model1: **{f1_m1}**\n")
md_writer.print_data(f"Accuracy Model1: **{cr1['accuracy']}**\n")

md_writer.print_data(f"F1 Score (Macro) Model2: **{f1_m2}**\n")
md_writer.print_data(f"Accuracy Model2: **{cr2['accuracy']}**\n")

md_writer.header2('Classification Report')

md_writer.print_data(report_df1.to_markdown(index=True) + '\n\n')
md_writer.print_data(report_df2.to_markdown(index=True) + '\n\n')

md_writer.header2('Confusion Matrix')
matrix_df1.index = [key for key, value in sorted(label_map_1.items(), key=lambda item: item[1])]
matrix_df1.columns = [key for key, value in sorted(label_map_1.items(), key=lambda item: item[1])]
md_writer.print_data(matrix_df1.to_markdown(index=True) + '\n')

md_writer.header2('Confusion Matrix Model 2')
matrix_df2.index = [key for key, value in sorted(label_map_2.items(), key=lambda item: item[1])]
matrix_df2.columns = [key for key, value in sorted(label_map_2.items(), key=lambda item: item[1])]
md_writer.print_data(matrix_df1.to_markdown(index=True) + '\n')

# Define the plot
plt.figure(figsize=(10, 8))
sns.heatmap(cm1, annot=True, fmt='d', cmap='Blues', xticklabels=list(label_map_1.keys()), yticklabels=list(label_map_1.keys()))
plt.title('Confusion Matrix Model 1')
plt.ylabel('Actual')
plt.xlabel('Predicted')

# Save the plot model1 as an image file
plt.savefig('report/confusion_matrix_m1.png', bbox_inches='tight')

plt.figure(figsize=(10, 8))
sns.heatmap(cm2, annot=True, fmt='d', cmap='Blues', xticklabels=list(label_map_2.keys()), yticklabels=list(label_map_2.keys()))
plt.title('Confusion Matrix Model 2')
plt.ylabel('Actual')
plt.xlabel('Predicted')

# Save the plot model2 as an image file
plt.savefig('report/confusion_matrix_m2.png', bbox_inches='tight')

md_writer.add_image('report/confusion_matrix_m1.png', 'Confusion matrix')
md_writer.add_image('report/confusion_matrix_m2.png', 'Confusion matrix')
# md_writer.add_image('dvctrain/static/eval/accuracy.png', 'Accuracy during the training')
# md_writer.add_image('dvctrain/static/eval/f1.png', 'F1Score during the training')
# md_writer.add_image('dvctrain/static/eval/loss.png', 'Loss during the training')
md_writer.write_toc('Report '+ params['model_mel_notmel']['arch']+" "+ params['model_mel_nv']['arch'])
